# **Assignment 1 - Logistic Regression**
## Author: Jake Brulato
## Tuesday 5:30 - 8:15, Kornelia

**Problem description and questions: A supermarket offers a new line of organic products. The
supermarket’s management wants to determine which customers are likely to purchase these
products. The supermarket has a customer loyalty program. As an initial buyer incentive plan,
the supermarket provided coupons for the organic products to all of the loyalty program
participants and collected data that includes whether these customers purchased any of the
organic products. Based on the data collected, the supermarket wants to understand the
behavior of their customers and their likelihood of purchase of organic products. The
ORGANICS data set contains 8 variables as shown in the table below and more than 22,000
observations**

In [2]:
#Import the packages you will use
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

In [3]:
#Import the data
df_JB = pd.read_csv('Organics.csv', sep=',')
df_JB

,ID,DemAffl,DemAge,DemGender,PromClass,PromSpend,PromTime,TargetBuy
0,140,10.0,76.0,U,Gold,16000.00,4.0,0
1,620,4.0,49.0,U,Gold,6000.00,5.0,0
2,868,5.0,70.0,F,Silver,0.02,8.0,1
3,1120,10.0,65.0,M,Tin,0.01,7.0,1
4,2313,11.0,68.0,F,Tin,0.01,8.0,0
...,...,...,...,...,...,...,...,...
22218,52834058,13.0,65.0,F,Silver,1500.00,5.0,0
22219,52834376,15.0,73.0,U,Gold,6053.06,12.0,0
22220,52837057,9.0,70.0,F,Gold,6000.00,5.0,0
22221,52838096,11.0,66.0,F,Silver,5000.00,5.0,0


In [4]:
#Drop the columns you wont be using for your data
# model_data_JB = df_JB.drop(columns= ['ID', 'PromClass'])
model_data_JB = df_JB.drop(columns= ['ID', 'DemGender'])
model_data_JB

,DemAffl,DemAge,PromClass,PromSpend,PromTime,TargetBuy
0,10.0,76.0,Gold,16000.00,4.0,0
1,4.0,49.0,Gold,6000.00,5.0,0
2,5.0,70.0,Silver,0.02,8.0,1
3,10.0,65.0,Tin,0.01,7.0,1
4,11.0,68.0,Tin,0.01,8.0,0
...,...,...,...,...,...,...
22218,13.0,65.0,Silver,1500.00,5.0,0
22219,15.0,73.0,Gold,6053.06,12.0,0
22220,9.0,70.0,Gold,6000.00,5.0,0
22221,11.0,66.0,Silver,5000.00,5.0,0


**1. What variable would you consider as the target variable? Explain your reason. (2 pts)**

I believe the target variable would be 'TargetBuy' as it is the only variable to return 0 or 1, with logistic regression, we want to predict the outcome of the binary variable based on the other independents in the model.

**2. Select any 5 variables to consider as independent variables in the model. Explain the
reasons for your selection. (5 pts)**

- PromClass: 
    - becuase it helps determine how often they go to the supermarket to be able to be a loyalty member, and likely being influence by the new products.
- PromSpend:
    - The amount of spending can show how likely they are to buy the new line of organic products, making a indicator on what wealth level people would be more likely to purchase it.
- PromTime:
    - The amount of time a person has been a member at the supermarket, could be impactful for a persons decision in buying organic.
- DemAffl:
    - How much a wealth a person has on a 30 point scales, shows what economic bracket people would be in to more likely buy the organic line.
- DemAge:
    - How old the person will be if they by the organics, could be a good indicator on what age range would try to eat healthier.


**3. Are there any variables which cannot be used in your model? Why? (3 pts)**

- ID would not make sense as it just serves as a unique identifier for each of the people's data in each row.
- DemGender could be used but it would be better to use age as a predictor as gender has less variability being limited to three classes.

**4. What variables need to be dummy coded before you run your logistic regression model?
Explain what new dummy coded columns you created. (6 pts)**

- PromClass would need to be dummy coded as logistic regression cannot take in text values. 

In [5]:
model_data_JB = pd.get_dummies(model_data_JB)
# model_data_JB['DemGender_F'] = model_data_JB['DemGender_F'].astype(int)
# model_data_JB['DemGender_M'] = model_data_JB['DemGender_M'].astype(int)
# model_data_JB['DemGender_U'] = model_data_JB['DemGender_U'].astype(int)
model_data_JB['PromClass_Gold'] = model_data_JB['PromClass_Gold'].astype(int)
model_data_JB['PromClass_Platinum'] = model_data_JB['PromClass_Platinum'].astype(int)
model_data_JB['PromClass_Silver'] = model_data_JB['PromClass_Silver'].astype(int)
model_data_JB['PromClass_Tin'] = model_data_JB['PromClass_Tin'].astype(int)
print(model_data_JB)
# print("Gold: ", model_data_JB['PromClass_Gold'].value_counts().get(1, 1))
# print("Plat: ", model_data_JB['PromClass_Platinum'].value_counts().get(1, 1))
# print("Silver: ", model_data_JB['PromClass_Silver'].value_counts().get(1, 1))
# print("Tin: ", model_data_JB['PromClass_Tin'].value_counts().get(1, 1))


       DemAffl  DemAge  PromSpend  PromTime  TargetBuy  PromClass_Gold  \
0         10.0    76.0   16000.00       4.0          0               1   
1          4.0    49.0    6000.00       5.0          0               1   
2          5.0    70.0       0.02       8.0          1               0   
3         10.0    65.0       0.01       7.0          1               0   
4         11.0    68.0       0.01       8.0          0               0   
...        ...     ...        ...       ...        ...             ...   
22218     13.0    65.0    1500.00       5.0          0               0   
22219     15.0    73.0    6053.06      12.0          0               1   
22220      9.0    70.0    6000.00       5.0          0               1   
22221     11.0    66.0    5000.00       5.0          0               0   
22222      3.0    75.0    6000.00       2.0          0               1   

       PromClass_Platinum  PromClass_Silver  PromClass_Tin  
0                       0                 0       

**5. Do you have to consider missing values in your dataset? How did you handle the presence
of missing values, if any? (4 pts)**

In [6]:
#Calculate the nulls and then drop them
model_data_JB.isnull().sum()
clean_JB = model_data_JB.dropna()
clean_JB.info()
print(clean_JB.isnull().sum())
clean_JB

<class 'pandas.core.frame.DataFrame'>
Index: 19471 entries, 0 to 22222
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   DemAffl             19471 non-null  float64
 1   DemAge              19471 non-null  float64
 2   PromSpend           19471 non-null  float64
 3   PromTime            19471 non-null  float64
 4   TargetBuy           19471 non-null  int64  
 5   PromClass_Gold      19471 non-null  int32  
 6   PromClass_Platinum  19471 non-null  int32  
 7   PromClass_Silver    19471 non-null  int32  
 8   PromClass_Tin       19471 non-null  int32  
dtypes: float64(4), int32(4), int64(1)
memory usage: 1.2 MB
DemAffl               0
DemAge                0
PromSpend             0
PromTime              0
TargetBuy             0
PromClass_Gold        0
PromClass_Platinum    0
PromClass_Silver      0
PromClass_Tin         0
dtype: int64


,DemAffl,DemAge,PromSpend,PromTime,TargetBuy,PromClass_Gold,PromClass_Platinum,PromClass_Silver,PromClass_Tin
0,10.0,76.0,16000.00,4.0,0,1,0,0,0
1,4.0,49.0,6000.00,5.0,0,1,0,0,0
2,5.0,70.0,0.02,8.0,1,0,0,1,0
3,10.0,65.0,0.01,7.0,1,0,0,0,1
4,11.0,68.0,0.01,8.0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...
22218,13.0,65.0,1500.00,5.0,0,0,0,1,0
22219,15.0,73.0,6053.06,12.0,0,1,0,0,0
22220,9.0,70.0,6000.00,5.0,0,1,0,0,0
22221,11.0,66.0,5000.00,5.0,0,0,0,1,0


**6. Provide following screenshots from your logistic regression model. (5 pts)**
- a. The model result summary, along with the coefficient table
- b. The classification (confusion) matrix output


In [7]:
y = np.array(clean_JB['TargetBuy'])

x = clean_JB[['DemAffl', 'DemAge', 'PromSpend', 'PromTime', 'PromClass_Gold', 'PromClass_Platinum', 'PromClass_Silver', 'PromClass_Tin' ]]
# x = clean_JB[['DemAffl', 'DemAge', 'PromSpend', 'PromTime', 'DemGender_M', 'DemGender_F', 'DemGender_U']]
X_train, X_test, y_train, y_test = train_test_split(x,y)

X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)

model = sm.Logit(y_train, X_train)
result = model.fit()

print(result.summary())

Optimization terminated successfully.
         Current function value: 0.455393
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                14603
Model:                          Logit   Df Residuals:                    14595
Method:                           MLE   Df Model:                            7
Date:                Tue, 30 Jan 2024   Pseudo R-squ.:                  0.1857
Time:                        21:39:29   Log-Likelihood:                -6650.1
converged:                       True   LL-Null:                       -8166.7
Covariance Type:            nonrobust   LLR p-value:                     0.000
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 -0.6229   1.19e+06  -5.25e-07      1.000   -2.33e+06    2.33e+06
DemAf

**7. Is the overall model statistically significant? Explain how you arrived at your conclusion. (4
pts)**

**8. Interpret the impacts of each independent variable on the target variable. (10 pts)**

**9. From the confusion matrix, compute the accuracy, precision, recall and F1 score for this
model. (6 pts)**

**10. Summarize your findings to the Director of Marketing for this company. Based on your
summary, provide two recommendations that could address the company’s problem
described earlier. (Limit this answer to a paragraph of not more than 200 words) (5 pts)**